# [Implementing Back Propagation](https://github.com/nfmcclure/tensorflow_cookbook/tree/master/02_TensorFlow_Way/05_Implementing_Back_Propagation)

One of the benefits of using TensorFlow, is that it can keep track of operations and automatically update model variables based on back propagation. In this recipe, we will introduce how to use this aspect to our advantage when training machine learning models.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

In [2]:
SAMPLE_SIZE = 800
BATCH_SIZE = 80
ITERATIONS = 10000
LOG_INTERVAL = 500

In [4]:
def getOptimizer(learning_rate):
    optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
#     optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#     optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.15)
#     optimizer = tf.train.ProximalAdagradOptimizer(learning_rate=learning_rate)
#     optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate=learning_rate)
    return optimizer

## Simple regression algorithm

In [ ]:
ops.reset_default_graph()
sess = tf.Session()

### Creating data, placeholders and variables

In [ ]:
x_vals = np.random.normal(loc=6.5, scale=1.2, size=SAMPLE_SIZE)
y_vals = np.vectorize(lambda x: x * 4.8 + np.random.normal(loc=0, scale=0.2))(x_vals)
print(x_vals[: 6])
print(y_vals[: 6])

In [ ]:
x = tf.placeholder(dtype=tf.float32, shape=[BATCH_SIZE], name='x')
y = tf.placeholder(dtype=tf.float32, shape=[BATCH_SIZE], name='y')
print(x)
print(y)

In [ ]:
w = tf.Variable(tf.truncated_normal(shape=[1]), name='w')
print(w)

### Calculating predictions and loss

In [ ]:
predictions = tf.multiply(w, x, name='predictions')
loss = tf.nn.l2_loss(predictions - y, name='loss')
print(predictions)
print(loss)

### Declaring optimizer and training step

In [ ]:
optimizer = getOptimizer(learning_rate=0.0005)
train = optimizer.minimize(loss)

### Initializing variables

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)

### Training

In [ ]:
for i in range(ITERATIONS):
    rand_idx = np.random.choice(a=SAMPLE_SIZE, size=BATCH_SIZE, replace=False)
    x_rand = x_vals[rand_idx]
    y_rand = y_vals[rand_idx]
    feedDict = {x: x_rand, y: y_rand}
    sess.run(train, feed_dict=feedDict)
    
    if i == 0 or (i + 1) % LOG_INTERVAL == 0:
        wRes, lossRes = sess.run([w, loss], feed_dict=feedDict)
        print('#{} w = {}, loss = {}'.format(i + 1, wRes, lossRes))

## Simple classification algorithm

In [14]:
ops.reset_default_graph()
sess = tf.Session()

### Creating data, placeholders and variables

In [15]:
half_size = SAMPLE_SIZE // 2
x_vals = np.concatenate((np.random.normal(loc=-1, scale=1, size=half_size),
                       np.random.normal(loc=3, scale=1, size=half_size)))

y_vals = np.concatenate((np.repeat(a=0, repeats=half_size),
                       np.repeat(a=1, repeats=half_size)))
print(x_vals[: 6])
print(y_vals[: 6])

[-0.78883276  1.0664738  -0.84926703 -2.65555492 -2.28857167 -1.30690461]
[0 0 0 0 0 0]


In [16]:
x = tf.placeholder(dtype=tf.float32, shape=[BATCH_SIZE, 1], name='x')
y = tf.placeholder(dtype=tf.float32, shape=[BATCH_SIZE, 1], name='y')
print(x)
print(y)

Tensor("x:0", shape=(80, 1), dtype=float32)
Tensor("y:0", shape=(80, 1), dtype=float32)


In [17]:
w = tf.Variable(tf.truncated_normal(mean=0, shape=[1]), name='w')
b = tf.Variable(tf.truncated_normal(mean=2, shape=[1]), name='b')
print(w)
print(b)

Tensor("w/read:0", shape=(1,), dtype=float32)
Tensor("b/read:0", shape=(1,), dtype=float32)


### Calculating outputs and loss

In [19]:
output = tf.add(tf.multiply(w, x), b, name='outputs')
xentropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=output), name='xentropy')
print(output)
print(xentropy)

Tensor("outputs_1:0", shape=(80, 1), dtype=float32)
Tensor("xentropy_1:0", shape=(), dtype=float32)


### Declaring optimizer and training step

In [20]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1)
train = optimizer.minimize(xentropy)

### Initializing variables

In [21]:
init = tf.global_variables_initializer()
sess.run(init)

### Training

In [22]:
for i in range(ITERATIONS):
    rand_idx = np.random.choice(a=SAMPLE_SIZE, size=BATCH_SIZE, replace=False)
    x_rand = np.expand_dims(a=x_vals[rand_idx], axis=1)
    y_rand = np.expand_dims(a=y_vals[rand_idx], axis=1)
    feedDict = {x: x_rand, y: y_rand}
    sess.run(train, feed_dict=feedDict)
    
    if i % LOG_INTERVAL == 0:
        wRes, bRes, xentropyRes = sess.run([w, b, xentropy], feed_dict=feedDict)
        print('#{} w = {}, b = {}, xentropy = {}'.format(i + 1, wRes, bRes, xentropyRes))

#1 w = [ 0.76009929], b = [ 1.32883883], xentropy = 0.4946681559085846
#501 w = [ 3.65443563], b = [-3.56057477], xentropy = 0.09474021196365356
#1001 w = [ 3.89725685], b = [-3.71176076], xentropy = 0.024227600544691086
#1501 w = [ 3.83745241], b = [-3.97193694], xentropy = 0.07012464851140976
#2001 w = [ 3.84695029], b = [-3.9284234], xentropy = 0.11310561001300812
#2501 w = [ 3.90799046], b = [-3.91635633], xentropy = 0.08712489902973175
#3001 w = [ 3.83545256], b = [-3.99198675], xentropy = 0.02540026605129242
#3501 w = [ 4.02067327], b = [-3.92227888], xentropy = 0.03159834071993828
#4001 w = [ 3.97994828], b = [-3.98675418], xentropy = 0.022220898419618607
#4501 w = [ 3.94023776], b = [-3.91804433], xentropy = 0.1165633350610733
#5001 w = [ 3.94992208], b = [-4.01708364], xentropy = 0.009918822906911373
#5501 w = [ 4.03207922], b = [-3.93867087], xentropy = 0.057833850383758545
#6001 w = [ 3.92850971], b = [-3.98208046], xentropy = 0.08034975826740265
#6501 w = [ 3.92189813], b =